In [1]:
import pandas as pd
import numpy as np
import nltk


In [2]:
annot_df = pd.read_csv('./coco-captions-2017.csv')

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 
  

In [57]:
annot_list = annot_df.values
data = [] 
data_set = set()

# iterate through each caption 
for row in annot_list: 
    temp = [] 
      
    # tokenize the captions into words 
    for j in word_tokenize(row[1]): 
        data_set.add(j.lower())
        temp.append(j.lower()) 
  
    data.append(temp) 
  


In [58]:
'brave' in data_set

True

In [62]:
w2v_model = Word2Vec(data, size=100, workers=4, max_vocab_size=None, min_count = 0)

In [70]:
w2v_model.most_similar('moon')

[('chandelier', 0.7942395210266113),
 ('aquariums', 0.7931094169616699),
 ('valley', 0.7802001237869263),
 ('neath', 0.7800871133804321),
 ('drum', 0.7732276916503906),
 ('stairway', 0.7671425342559814),
 ('chimney', 0.7635893821716309),
 ('seravel', 0.7623745203018188),
 ('crane', 0.7557861804962158),
 ('rises', 0.7513242959976196)]

In [71]:
w2v_model.save('./text_encoding.bin')

In [5]:

data_cleaned = []
for row in data:
    if len(' '.split(row)) > 20:
        continue
    data_cleaned.append(row.lower())

In [6]:
# doc_tags = []
# for i in range(len(data_cleaned)):
#     doc_tags.append(i)
from gensim.models.doc2vec import TaggedDocument

docs = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(data_cleaned)]

In [36]:
# Bag of words model
# model1 = gensim.models.Word2Vec(data, min_count = 1,  
                             # size = 100, window = 5) 
# Skip Gram model 
# model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, 
                                # window = 5, sg = 1) 

# model_doc_100 = gensim.models.Doc2Vec(docs, vector_size=100, workers=8, dm=1, max_vocab_size=None, dm_mean=0)
model_doc_sum = gensim.models.Doc2Vec(docs, vector_size=256, workers=8, dm=1, max_vocab_size=None, dm_mean=0, min_count=10)
# model_doc_500 = gensim.models.Doc2Vec(docs, vector_size=500, workers=8, dm=1, max_vocab_size=None, dm_mean=0)
# model_doc_mean = gensim.models.Doc2Vec(docs, vector_size=256, workers=8, dm=1, max_vocab_size=None, dm_mean=1, min_count=10)

# dm_mean=1 enables vector averaging versus vector summing. Further testing required.
# model_doc = gensim.models.Doc2Vec(docs, vector_size=100, min_count=5, workers=8, dm=1, max_vocab_size=None, dm_mean=1)

In [37]:
model_doc_sum.save('textencodingmodel.bin')

In [33]:
s1= 'A woman uses a paintbrush on a melting candle.'
teststr = word_tokenize(s1)


teststr = [x.lower() for x in teststr]
# infer_vector = model_doc.infer_vector(teststr, steps=20000, alpha=0.025)
# similar_documents = model_doc.docvecs.most_similar([infer_vector], topn = 10)
# model_doc.docvecs.similarity_unseen_docs(model_doc, \
#                                          ['a', 'truck', 'loaded', 'down', 'with', 'bags', 'of', 'cans', 'in', 'the', 'bed', 'of', 'the', 'truck'], \
#                                          ['a', 'truck', 'bed', 'loaded', 'down'], \
#                                          steps=100000)

# model_doc.docvecs.similarity_unseen_docs(model_doc, \
#                                          ['fresh', 'apple', 'juice'], \
#                                          ['gross', 'apple', 'juice'], \
#                                          steps=10000)

num_outputs = 5

print('\n'+s1+'\n')
print('\nMean\n')
inf_vec = model_doc_mean.infer_vector(teststr, steps=10000)#, alpha=1, min_alpha=0.002)
output = model_doc_mean.docvecs.most_similar([inf_vec], topn=num_outputs)
# for doc in similar_documents:
#     print(data_cleaned[int(doc[0])])

for i in range(num_outputs):    
    print (data_cleaned[(int(output[i][0]) + 1)])
    print (output[i][1])
    
print('\nSum\n')
inf_vec = model_doc_sum.infer_vector(teststr, steps=10000)#, alpha=1, min_alpha=0.002)
output = model_doc_sum.docvecs.most_similar([inf_vec], topn=num_outputs)

for i in range(num_outputs):    
    print (data_cleaned[(int(output[i][0]) + 1)])
    print (output[i][1])
output    

    



A woman uses a paintbrush on a melting candle.


Mean

a baseball player holding a bat over his shoulder.
0.6386563181877136
a bag that is filled with pens and scissors
0.634752094745636
a small airplane flying above a snow covered mountain.
0.6334839463233948
a narrow kitchen with a refrigerator at the end of it.
0.6208945512771606
a bus with a bike attached to the front is traveling down a street.
0.6205453276634216

Sum

toothbrushes are arranged in a cup on the bathroom counter.
0.6886410713195801
a group of birds standing around in a field. 
0.6870393753051758
workers cleaning a large green and white boat
0.6720724105834961
a little girl in a pink jacket riding a horse on the beach.
0.6626769304275513
a pile of assorted vegetables sitting side by side.
0.6622759103775024


[('35665', 0.6886410713195801),
 ('58802', 0.6870393753051758),
 ('70641', 0.6720724105834961),
 ('75843', 0.6626769304275513),
 ('81955', 0.6622759103775024)]

5

In [249]:
model_doc.docvecs.similarity_unseen_docs(model_doc, \
                                         ['fresh', 'apple', 'juice'], \
                                         ['fresh', 'apple', 'juice'], \
                                         steps=10000)

0.9549301027985857

In [276]:
model_doc.most_similar(['wiimote'])

[('begin', 0.81293785572052),
 ('tag', 0.7691446542739868),
 ('runner', 0.7600712180137634),
 ('read', 0.7600324153900146),
 ('student', 0.7563343644142151),
 ('own', 0.7557961940765381),
 ('balance', 0.7526955604553223),
 ('she', 0.7499886155128479),
 ('listening', 0.7498931884765625),
 ('see', 0.7497900724411011)]